In [ ]:
# [1] COLLECT ALL GROUND TRUTH DATA
from parse_truth_ALL import main as parse_truth_all
import collections
from plot_overlap_ratios import plot_ratio_success
from algorithm_output_parser import process_algorithm_output
from algorithm_output_parser import toboggan_output_parser
from algorithm_output_parser import catfish_output_parser

froots = ['human', 'mouse', 'zebra']
show_flag = False

In [14]:
gt_pathset_dict = []
all_catfish_paths = []
all_toboggan_paths = []
all_stats = [collections.defaultdict(list), collections.defaultdict(list), collections.defaultdict(list)]
indices_identical = [collections.defaultdict(list), collections.defaultdict(list), collections.defaultdict(list)]
indices_different = [collections.defaultdict(list), collections.defaultdict(list), collections.defaultdict(list)]
indices_timeout = [collections.defaultdict(list), collections.defaultdict(list), collections.defaultdict(list)]

for which_dataset in [0,1,2]:
    datadirending = ['human', 'mouse', 'zebrafish'][which_dataset]
    datadir = '/home/kyle/data/rnaseq/' + datadirending + '/'
    froot = froots[which_dataset]
    
    # Given list of gt filename-instancenum pairs, aggregate dictionary:
    gt_pathset_dict = parse_truth_all(datadir)

    # [2] GET CATFISH INFO
    catfish_results_file = '../catfish-comparison/catfish-log-' + froot + '.txt'
    all_catfish_paths = catfish_output_parser(catfish_results_file, gt_pathset_dict, verbose=False)

    # [3] GET TOBOGGAN INFO
    toboggan_results_file = './data/' + froot + '-master-file.txt'
    all_toboggan_paths = toboggan_output_parser(toboggan_results_file, gt_pathset_dict, verbose=False)

    # [4] COMPARE GROUND TRUTH VS ALGORITHM OUTPUT
    # FILTER ALGORITHM OUTPUTS FOR RELEVANT DATA

    j = which_dataset
    all_stats[j][0], indices_identical[j][0], indices_different[j][0], indices_timeout[j][0] = process_algorithm_output(gt_pathset_dict, all_catfish_paths)
    all_stats[j][1], indices_identical[j][1], indices_different[j][1], indices_timeout[j][1] = process_algorithm_output(gt_pathset_dict, all_toboggan_paths)

Num timeouts = 0
Num timeouts = 2473
Num timeouts = 0
Num timeouts = 2485
Num timeouts = 0
Num timeouts = 178


In [60]:
import matplotlib.pyplot as plt
import numpy
figurename = 'size-vs-ratio-identical-'

def plot_ratios(figurename, froot, all_stats, indices_identical, indices_timeout, show_flag=True):
    plt.close()
    plt.clf()
    plt.figure(figsize=(4,3))

    alg_names = ['catfish', 'toboggan']
    colors = ['red', 'blue']

    def get_width(num_datapoints):
        return 0.18*numpy.log10(num_datapoints)

    fig, ax1 = plt.subplots()
    
    fig_alg = {}
    max_label = 0

    for which_alg in range(len(alg_names)):
        print(" {} with # data points {} ".format(alg_names[which_alg], len(all_stats[which_alg])) )
        # all_stats.append( [len(gt_pathset), len(catfish_pathset), this_jaccard_value, this_intersect, gap] )
        stats = all_stats[which_alg]
        inds_identical = indices_identical[which_alg]
        inds_timeouts = indices_timeout[which_alg]

        data1 = numpy.array(stats[:,0])  # size of ground truth pathset
        data2 = numpy.array(stats[inds_identical, 0])  # size of ground truth pathset when algorithm identical

        data1_counter = collections.Counter([ x[0] for x in data1 ])
        data2_counter = collections.Counter([ x[0] for x in data2 ])

        data1_counter = collections.Counter([ x[0] for x in data1 ])
        data2_counter = collections.Counter([ x[0] for x in data2 ])

        data = []
        soln_sizes = []
        num_datapoints = []
        widths = []
        for soln_size, x in data1_counter.items():
            y = data2_counter[soln_size]
            if x != 0:
                data.append(y/x)
                num_datapoints.append(int(x))
                soln_sizes.append(int(soln_size))
                widths.append(get_width(x))
                if y != 0:
                    max_label = max( max_label, int(soln_size) )

        ax1.scatter(soln_sizes, data, color=colors[which_alg], label=alg_names[which_alg], alpha=0.5)
    
    axis_font_size = 15
    axis_font = {'fontname':'DejaVu Sans', 'size':'{}'.format(axis_font_size)}
    title_font = {'fontname':'DejaVu Sans', 'size':'16'}
    xtick_font = 12 #{'fontname':'DejaVu Sans', 'size':'11'}
    ytick_font = 13 #{'fontname':'DejaVu Sans', 'size':'12'}
    #         plt.ylabel("Frequency", **axis_font)
    #         plt.gca().set_yticklabels([10**x for x in range(0, 6)], **ytick_font)
    #     plt.title("dataset: {}".format(froot), **title_font)
    #     plt.xlabel("time", **axis_font)
    #     plt.gca().set_xticklabels([10**x for x in range(-4, 5)], **xtick_font)

    
    
    
    plt.title("Pathset overlap on {}".format(froot), **title_font)
    plt.xlabel("Ground truth decomposition size", **axis_font)
    xlims = plt.xlim()
    ax1.set_xlim( [ xlims[0], 21])
    ax1.set_ylabel("Proportion of instances: output = GT", color='purple', **axis_font)
    ax1.set_ylim([0,1])
    ax1.tick_params('y', colors='purple', labelsize=ytick_font)

    # get all ground truth sizes
    soln_sizes = [ (int(soln_size), int(count)) for soln_size, count in \
                  collections.Counter([ x[0] for x in \
                                       numpy.array(all_stats[0][:,0]) ]).items() if count != 0 ]         
    soln_sizes, num_datapoints = zip(*soln_sizes)

    plt.xticks( range(1, int(max(soln_sizes)+1)) )
    ax2 = ax1.twinx()
    ydata = numpy.array(num_datapoints)
    
    ax2.bar(list(soln_sizes), list(ydata/sum(ydata)), color='grey', alpha=0.33)
    ax2.set_ylabel('Ratio of all instances', color='grey', **axis_font)
    ax2.set_xlim( [xlims[0], 21])
    #     ax2.set_yscale("log")
    ax2.tick_params('y', colors='grey', labelsize=ytick_font)

    ax1.legend(loc='upper right', frameon=False, prop={'size':axis_font_size})#, handles=ax1, labels=alg_names)
    fig.tight_layout()

    if show_flag:
        plt.show()
    else:
        plt.savefig( './figures/' + ''.join([figurename, froot, '.pdf']), format='pdf', dpi=300)
        plt.savefig( './figures/' + ''.join([figurename, froot, '.png']), format='png', dpi=300)
    plt.close()
    print("Done with plot {}".format(figurename))

In [61]:

for which_data in [0,1,2]:
    froot = froots[which_data]
    plot_ratios(figurename, froot, all_stats[which_data], indices_identical[which_data], indices_timeout[which_data], show_flag=False)

 catfish with # data points 529523 
 toboggan with # data points 529523 
Done with plot size-vs-ratio-identical-
 catfish with # data points 473185 
 toboggan with # data points 473185 
Done with plot size-vs-ratio-identical-
 catfish with # data points 445880 
 toboggan with # data points 445880 
Done with plot size-vs-ratio-identical-
